<a href="https://colab.research.google.com/github/sandeep1214/ANDROID-/blob/main/smart_campus_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sqlite3
import random
from datetime import date, timedelta

# -------------------------------
# Create SQLite database
# -------------------------------
conn = sqlite3.connect("smart_campus_energy.db")
cursor = conn.cursor()
# -------------------------------
# Create tables with constraints
# -------------------------------
cursor.executescript("""
DROP TABLE IF EXISTS UsageLogs;
DROP TABLE IF EXISTS EnergyUsage;
DROP TABLE IF EXISTS Buildings;

CREATE TABLE Buildings (
    building_id INTEGER PRIMARY KEY,
    building_name TEXT,
    building_type TEXT CHECK (
        building_type IN ('Academic', 'Residential', 'Laboratory', 'Administrative')
    ),
    department TEXT,
    building_area_sqm REAL CHECK (building_area_sqm > 0),
    sustainability_rating TEXT CHECK (
        sustainability_rating IN ('Poor', 'Fair', 'Good', 'Excellent')
    )
);

CREATE TABLE EnergyUsage (
    usage_id INTEGER PRIMARY KEY,
    building_id INTEGER,
    energy_source TEXT CHECK (
        energy_source IN ('Grid', 'Solar', 'Wind')
    ),
    energy_consumption_kwh REAL CHECK (energy_consumption_kwh >= 0),
    water_usage_liters REAL CHECK (water_usage_liters >= 0),
    FOREIGN KEY (building_id) REFERENCES Buildings(building_id)
);

CREATE TABLE UsageLogs (
    building_id INTEGER,
    log_date TEXT,
    temperature_celsius REAL,
    occupancy_level TEXT CHECK (
        occupancy_level IN ('Low', 'Medium', 'High')
    ),
    air_quality_index INTEGER,
    PRIMARY KEY (building_id, log_date),
    FOREIGN KEY (building_id) REFERENCES Buildings(building_id)
);
""")

# -------------------------------
# Reference data
# -------------------------------
building_types = ["Academic", "Residential", "Laboratory", "Administrative"]
departments = ["Engineering", "Science", "Business", "Arts"]
ratings = ["Poor", "Fair", "Good", "Excellent"]
energy_sources = ["Grid", "Solar", "Wind"]
occupancy_levels = ["Low", "Medium", "High"]

# -------------------------------
# Insert Buildings (40 rows)
# -------------------------------
for i in range(1, 41):
    cursor.execute("""
        INSERT INTO Buildings VALUES (?, ?, ?, ?, ?, ?)
    """, (
        i,
        f"Building_{i}",
        random.choice(building_types),
        random.choice(departments),
        round(random.uniform(500, 5000), 2),      # Ratio data
        random.choice(ratings)                    # Ordinal data
    ))

# -------------------------------
# Insert Energy Usage (40 rows)
# -------------------------------
usage_id = 1
for building_id in range(1, 41):
    cursor.execute("""
        INSERT INTO EnergyUsage VALUES (?, ?, ?, ?, ?)
    """, (
        usage_id,
        building_id,
        random.choice(energy_sources),             # Nominal data
        round(random.uniform(1000, 10000), 2),     # Ratio data
        round(random.uniform(5000, 50000), 2)      # Ratio data
    ))
    usage_id += 1

# -------------------------------
# Insert Usage Logs (1200+ rows)
# -------------------------------
start_date = date(2025, 1, 1)

for building_id in range(1, 41):
    for day in range(30):  # 40 buildings × 30 days = 1200 rows
        log_date = start_date + timedelta(days=day)
        cursor.execute("""
            INSERT INTO UsageLogs VALUES (?, ?, ?, ?, ?)
        """, (
            building_id,
            log_date.isoformat(),
            round(random.uniform(15, 30), 2),       # Interval data
            random.choice(occupancy_levels),        # Ordinal data
            random.randint(50, 150)                 # Interval data
        ))

# -------------------------------
# Save and close
# -------------------------------
conn.commit()
conn.close()

print("Database 'smart_campus_energy.db' created successfully.")


Database 'smart_campus_energy.db' created successfully.
